In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
import os
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By

from time import sleep
from dotenv import load_dotenv
load_dotenv()

In [ ]:
os.environ['EMAIL'], os.environ['PASSWORD']

In [ ]:
driver = webdriver.Chrome()

In [ ]:
driver.get('https://www.linkedin.com/login')

In [ ]:
driver.title

In [ ]:
email = driver.find_element(By.ID, 'username')
email.send_keys(os.environ['EMAIL'])

password = driver.find_element(By.ID, 'password')
password.send_keys(os.environ['PASSWORD'])

password.submit()

In [ ]:
url = "https://www.linkedin.com/in/vinit-raj-singh-0312b5286/"
driver.get(url)

In [ ]:
profile_data = {}

In [ ]:
driver.title

In [ ]:
page_source = driver.page_source #i am using beautifulsoup to get the acess to html content 
soup = BeautifulSoup(page_source, 'lxml')

name = soup.find('h1', {'class': 'JaiUrdAKgwlLZfbukVHeAbzmRQqGaKXozY inline t-24 v-align-middle break-words'})
name = name.get_text().strip()

profile_data['name'] = name
profile_data['url'] = url

In [ ]:
profile_data

In [ ]:
headline = soup.find('div', {'class': 'text-body-medium break-words'})
headline = headline.get_text().strip()

profile_data['headline'] = headline

In [ ]:
profile_data

### show more

In [ ]:
driver.find_element(By.CLASS_NAME, 'inline-show-more-text__button').click()

In [ ]:
page_source = driver.page_source 
soup = BeautifulSoup(page_source, 'lxml')

about = soup.find('div', {'class': 'display-flex ph5 pv3'})
about = about.get_text().strip()

profile_data['about'] = about

In [ ]:
profile_data

### Read Experience Section

In [ ]:
page_source = driver.page_source
soup = BeautifulSoup(page_source, 'lxml')

sections = soup.find_all('section', {'class': 'artdeco-card pv-profile-card break-words mt2'})

In [ ]:
for sec in sections:
    if sec.find('div', {'id': 'experience'}):
        experience = sec

#print(experience.get_text().strip())    #its giving vague text because we have not parsed the html,  we have to pretify it

In [ ]:
experience = experience.find_all('div', {'class': 'RcAGfcvUYOwPnRPDghLUditBqlesQtLJAck YOHdsXEIguVUsMOgTIJuvGTfijGweIQfbFPo cqNvkriDBrbznvYCmjFgzuCaXycArNac'})


In [ ]:
len(experience)  

In [ ]:
exp = experience[0]     # first experience

In [ ]:
def get_exp(exp):
        exp_dict = {}

        exp_name = exp.find('div', {"class": "display-flex flex-wrap align-items-center full-height"})  # this gives whole div inside which we have two span tags in which we find the visually hidden one
        exp_name = exp_name.find('span', {'class': 'visually-hidden'})
        exp_name = exp_name.get_text().strip()

        duration = exp.find('span', {"class": "t-14 t-normal"})  # this gives whole div inside which we have two span tags in which we find the visually hidden one
        duration = duration.find('span', {'class': 'visually-hidden'})
        duration = duration.get_text().strip()
        duration

        exp_dict['company_name/designation'] = exp_name
        exp_dict['duration/company_name'] = duration

        designations = exp.find_all('div', {'class': 'RcAGfcvUYOwPnRPDghLUditBqlesQtLJAck'})

        item_list = []
        for position in designations:
            spans = position.find_all('span', {'class': 'visually-hidden'})

            item_dict = {}
            item_dict['designation'] = spans[0].get_text().strip()
            item_dict['duration'] = spans[1].get_text().strip()

            try:
                item_dict['location'] = spans[2].get_text().strip()
            except:
                item_dict['location'] = ""

            try:
                item_dict['type'] = spans[3].get_text().strip()
            except:
                item_dict['type'] = ""

            try:
                item_dict['description'] = spans[4].get_text().strip()
            except:
                item_dict['description'] = ""

            item_list.append(item_dict)

        exp_dict['designations'] = item_list
        return exp_dict

item_list = []
for exp in experience:
    item_list.append(get_exp(exp))

In [ ]:
profile_data['experience'] = item_list

In [ ]:
profile_data

### Education

In [ ]:
for sec in sections:
    if sec.find('div', {'id': 'education'}):
        educations = sec


In [ ]:
# education.get_text().strip()

In [ ]:
items = educations.find_all('div', {'class': 'RcAGfcvUYOwPnRPDghLUditBqlesQtLJAck YOHdsXEIguVUsMOgTIJuvGTfijGweIQfbFPo cqNvkriDBrbznvYCmjFgzuCaXycArNac'})

In [ ]:
len(items)

In [ ]:
def get_edu(item):
    item_dict = {}
    spans = item.find_all('span', {'class': 'visually-hidden'})

    item_dict['college'] = spans[0].get_text().strip()
    item_dict['degree'] = spans[1].get_text().strip()
    item_dict['duration'] = spans[2].get_text().strip()
    item_dict['project/grade'] = spans[3].get_text().strip()

    return item_dict

item_list = []
for item in items:
    item_list.append(get_edu(item))

profile_data['education'] = item_list

In [ ]:
profile_data

### Licenses & Certifications

In [ ]:
driver.find_element(By.ID, 'navigation-index-see-all-licenses-and-certifications').click()    # for showing all the certifications same as see all in the about section

In [ ]:
page_source = driver.page_source 
soup = BeautifulSoup(page_source, 'lxml')

soup = soup.find('section', {'class': 'artdeco-card pb3'})  

In [ ]:
items = soup.find_all('div', {'class': 'RcAGfcvUYOwPnRPDghLUditBqlesQtLJAck YOHdsXEIguVUsMOgTIJuvGTfijGweIQfbFPo cqNvkriDBrbznvYCmjFgzuCaXycArNac'})

len(items)

In [ ]:
item = items[0]

def get_license(item):
    spans = item.find_all('span', {'class': 'visually-hidden'})

    item_dict = {}
    item_dict['name'] = spans[0].get_text().strip()
    item_dict['institute'] = spans[1].get_text().strip()
    item_dict['issued_date/Skills'] = spans[2].get_text().strip()

    return item_dict

item_list = []
for item in items:
    item_list.append(get_license(item))

profile_data['licenses'] = item_list

In [ ]:
profile_data

In [ ]:
driver.back()

In [ ]:
import json

In [ ]:
import json

with open('data/profile_data.json', 'w') as f:
    json.dump(profile_data, f, indent=4)

In [ ]:
driver.back()

In [ ]:
import csv

def flatten_profile_data(profile_data):
    flat = {
        'name': profile_data.get('name', ''),
        'url': profile_data.get('url', ''),
        'headline': profile_data.get('headline', ''),
        'about': profile_data.get('about', '')
    }
    # Experience
    experiences = profile_data.get('experience', [])
    flat['experience'] = '|'.join([
        f"{exp.get('company_name/designation', '')} ({exp.get('duration/company_name', '')}) " +
        ', '.join([f"{d.get('designation', '')} [{d.get('duration', '')}]" for d in exp.get('designations', [])])
        for exp in experiences
    ])
    # Education
    educations = profile_data.get('education', [])
    flat['education'] = '|'.join([
        f"{edu.get('college', '')} - {edu.get('degree', '')} ({edu.get('duration', '')}) [{edu.get('project/grade', '')}]"
        for edu in educations
    ])
    # Licenses
    licenses = profile_data.get('licenses', [])
    flat['licenses'] = '|'.join([
        f"{lic.get('name', '')} - {lic.get('institute', '')} ({lic.get('issued_date/Skills', '')})"
        for lic in licenses
    ])
    return flat

flat_profile = flatten_profile_data(profile_data)

with open('data/profile_data.csv', 'w', newline='', encoding='utf-8') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=flat_profile.keys())
    writer.writeheader()
    writer.writerow(flat_profile)

print('Profile data saved to data/profile_data.csv')